In [3]:
import clinicalPicture as cp
import matplotlib.pyplot as plt
from itertools import product

In [4]:
PD_COUNT_MAX = 5

In [6]:
def createArayDelimiters(original_array_length, sample_lenght):
    if sample_lenght == 0:
        return [[]]
    stop = [*range(original_array_length-sample_lenght, original_array_length)]
    arr = []
    def create(word, i_let):
        while word[i_let] < stop[i_let]+1:
            if len(word) < sample_lenght:
                create([*word, word[-1]+1], i_let+1)
            else:
                arr.append(word.copy())
            word[i_let] += 1
        return
    create([1], 0)
    
    return arr

In [7]:
def findePosiblePD(ex: tuple[list, list]) -> tuple[ list, list[dict['zdp':, 'pd_duration': list[tuple[int]]]] ]:
    arr_val = ex[1]
    arr_time = ex[0]
    periods_dinamic = {f'{pd_count+1}':[] for pd_count in range(PD_COUNT_MAX)}
    for count_dilimetrs_pd in range(PD_COUNT_MAX):
        
            
        delimiters_arr = createArayDelimiters(len(arr_val), count_dilimetrs_pd)

        for delimiters in delimiters_arr:
            
            delimiters.append(None)
            delimiter_prev = 0
            v = []
            
            for delimiter in delimiters:
                v.append(set(arr_val[delimiter_prev:delimiter]))
                delimiter_prev = delimiter
                # if delimiters == [3, 6, 7,8 ,None]:
                #     print(v)
                if len(v)<2:
                    continue
                
                union_pd = v[-1] & v[-2]
                if len(union_pd) != 0:
                    break
            else:
                delimiter_prev = 0
                time = []
                for delimiter in delimiters:
                    times = arr_time[delimiter_prev:delimiter]
                    delimiter_prev = delimiter
                    time.append((times[0], times[-1]))
                    
                periods_dinamic[f'{count_dilimetrs_pd+1}'].append({'zdp': v, 'pd_duration': time})
    return periods_dinamic


In [8]:
def unionPD(pd1, pd2):

    def unionVal(arr1, arr2):
        ans = []
        for a, b in zip(arr1, arr2):
            ans.append(a | b)
            if len(ans)<2:
                continue
            
            union_pd = ans[-1] & ans[-2]
            if len(union_pd) != 0:
                return None
            
        return ans
    def unionTime(arr1, arr2):
        ans = []
        for a, b in zip(arr1, arr2):
            all_val = [*a, *b]
            ans.append((min(all_val), max(all_val)))
        return ans
    ans = {}
    for count_pd in range(1, PD_COUNT_MAX+1):
        ans[f'{count_pd}'] = []
        for combinations in product(pd1[f'{count_pd}'], pd2[f'{count_pd}']):
            union_val = unionVal(combinations[0]['zdp'], combinations[1]['zdp'])
            if union_val is not None:
                union_time = unionTime(combinations[0]['pd_duration'], combinations[1]['pd_duration'])
                ans[f'{count_pd}'].append({'zdp': union_val, 'pd_duration': union_time})
    return ans

In [ ]:
g = cp.ClinicalPicture(signs_count=2, disease_count=1)
g.generateSings(value_exponent_boundaries=[1, 3],sings_discrete_part=1,value_count_max=20)
g.generateDisease()

In [9]:
ex = g.createExample()

[[([9, 15, 16, 24, 25, 54, 58], [1, 1, 1, 0, 0, 1, 0]),
  ([4, 8, 20, 37, 40, 43, 57, 61], [4, 16, 12, 15, 19, 2, 1, 7])]]